The goal of this script is to eliminate compartment redundancy. Right now, there are 4 compartments: c, e, cytosol, and extracellular. We need to cut this down to just c and e so that the model can better account for the flow of metabolites into and out of the cell. 

In [1]:
# Initialization:
import cobra
import cobra.test
from cobra.flux_analysis import (single_gene_deletion)
import os
from os.path import join
import pandas
from time import time
import csv

# Load LJD functions which includes changeMedia_PA_LJD(model, media)
from LJD_Functions import *

# Had to install libsbml to load an xml file
# pip install python-libsbml in Python/Scripts folder
# website: http://sbml.org/Software/libSBML/Downloading_libSBML#Using_pip_from_PyPI
import libsbml

# Other packages
from copy import *
import scipy

In [2]:
# Load the model 
model_start = cobra.io.read_sbml_model('iPAU.xml')

In [3]:
# See compartments 
print(model_start.compartments)
e = 0
c = 0
cy = 0
ex = 0
for i in range(0, len(model_start.metabolites)):
    if model_start.metabolites[i].compartment == 'e':
        e = e + 1
    elif model_start.metabolites[i].compartment == 'c':
        c = c + 1
    elif model_start.metabolites[i].compartment == 'cytosol':
        cy = cy + 1
    elif model_start.metabolites[i].compartment == 'extracellular':
        ex = ex + 1
    else:
        print('other compartment', model_start.metabolites[i].compartment)
print('c', c)
print('e', e)
print('cytosol',cy)
print('extracellular',ex)
print('total', c+e+ex+cy, len(model_start.metabolites))

# Most of the metabolites seem to fall withing the default cytosol and extracellular compartments. This is also consistent with
# cobrapy so I will make them the default compartments. Especially since e = extracellular space...


{'cytosol': 'default', 'c': 'cytosol', 'extracellular': 'default', 'e': 'extracellular space'}
c 413
e 14
cytosol 681
extracellular 187
total 1295 1295


In [4]:
# Change media to LB
model_end = changeMedia_PA_LJD(model_start, 1) #change model name here
# Check that model is actually on LB
sol_LB = model_end.optimize()
print('LB', sol_LB.objective_value)

There is an `e` compartment but it does not look like it is the actual external compartment.


LB 16.51651595130235


In [5]:
# Try to change the compartment of a metabolite...
for x in range(0,len(model_end.metabolites)):
    if model_end.metabolites[x].compartment == 'e':
        model_end.metabolites[x].compartment = 'extracellular'
    elif model_end.metabolites[x].compartment == 'c':
        model_end.metabolites[x].compartment = 'cytosol'

# Check that this worked
e = 0
c = 0
cy = 0
ex = 0 
other = 0
for i in range(0, len(model_end.metabolites)):
    if model_end.metabolites[i].compartment == 'e':
        e = e + 1
    elif model_end.metabolites[i].compartment == 'c':
        c = c + 1
    elif model_end.metabolites[i].compartment == 'cytosol':
        cy = cy + 1
    elif model_end.metabolites[i].compartment == 'extracellular':
        ex = ex + 1
    else:
        other = other + 1
print('c', c)
print('e', e)
print('cytosol',cy)
print('extracellular',ex)
print('other',other)
print('total', c+e+ex+cy+other, len(model_end.metabolites))
print(model_end.compartments)


c 0
e 0
cytosol 1094
extracellular 201
other 0
total 1295 1295
{'cytosol': 'default', 'extracellular': 'default'}


In [ ]:
sol_LB = model_end.optimize()
print('LB', sol_LB.objective_value)

In [ ]:
# Export the model as xml and yml
cobra.io.write_sbml_model(model_end, "iPAU.xml")
cobra.io.save_yaml_model(model_end, "iPAU.yml")